# Jupyter Notebook UI to graph your TAP data!

Version 1.7 - Joseph Liang, Rankin Lab
Updated:
1. Upgraded folder path selection application
2. Upgraded dataset management (less moving parts for end-user)
3. output changed from tif -> png
4. Added tap-correction function for missing taps (should be working now)
5. Added plate column to evaluate plate-discrepancies
6. Added separate graphing function for plate-discrepancies
7. Added colour palette-setting function

## Known bug: Step 2 an empty windows displays in Mac. May also apply to linux/windows.

## Beginner Essentials:
1. Shift-Enter to run each cell. After you run, you should see an output "done step #". If not, an error has occured
2. When inputting your own code/revising the code, make sure you close all your quotation marks '' and brackets (), [], {}.
3. Don't leave any commas (,) hanging! (make sure an object always follows a comma. If there is nothing after a comma, remove the comma!
4. Learning to code? Each line of code is annotated to help you understand how this code works!

## 3. Run all cells/steps sequentially, even the ones that do not need input

## Steps that require input: #3, #6.1, #7

# 1. Importing Packages Required (No input required, just run)

In [1]:
import pandas as pd #<- package used to import and organize data
import numpy as np #<- package used to import and organize data
import math
import os #<- package used to work with system file paths
import seaborn as sns #<- package used to plot graphs
from matplotlib import pyplot as plt #<- another package used to plot graphs
from itertools import cycle #<- package used to iterate down rows (used in step 5 to add tap column)
import ipywidgets as widgets #<- widget tool to generate button and tab for graphs
from IPython.display import display #<- displays widgets
from tkinter import Tk, filedialog #<- Tkinter is a GUI package
print("done step 1")

done step 1


# 2. Pick filepath (just run and click button from output)

Run the following cell and click the button 'Select Folder' to pick a filepath.

## Important: Later on, this script uses the total file path for each file to import and group data. That means if your folder has whatever your strain is named, the script will not work.

(ex. if your folder has "N2" in it this script sees all files inside this folder as having the "N2" search key)

## An easy fix is to just rename your folder to something else (make your strains lower-case, or just have the date)

In [2]:
### Select Folder App - After you run, you can select your folder for filepath
button = widgets.Button(description = 'Select Folder') #<- Creates a button variable
display(button) #<- displays this button on output
def select_folder(b): #<- This is an action. Requires a variable, so I put in an arbitrary one 'b'
    global folder_path #<- sets a variable as a global variable, not just within this action
    Tk().withdraw() #<- Tkinter likes to create annoying empty windows. This removes them
    folder_path = filedialog.askdirectory() #<- Opens up a file explorer window, and determines folder path
    Tk().update()#<- below
    Tk().destroy()#<- this and the line above it removes the file explorer window after a selection is made
    print(folder_path) #<- this helps confirm that this action was performed
    print('done step 2')
button.on_click(select_folder) #<- describes what happens when you click on this button

Button(description='Select Folder', style=ButtonStyle())

1   HIToolbox                           0x0000000192c685c8 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 120
2   HIToolbox                           0x0000000192c68188 _ZN15MenuBarInstance14EnableAutoShowEv + 60
3   HIToolbox                           0x0000000192c0b310 SetMenuBarObscured + 372
4   HIToolbox                           0x0000000192c0aee8 _ZN13HIApplication15HandleActivatedEP14OpaqueEventRefhP15OpaqueWindowPtrh + 172
5   HIToolbox                           0x0000000192c04fcc _ZN13HIApplication13EventObserverEjP14OpaqueEventRefPv + 296
6   HIToolbox                           0x0000000192bcbcd0 _NotifyEventLoopObservers + 176
7   HIToolbox                           0x0000000192c0496c AcquireEventFromQueue + 432
8   HIToolbox                           0x0000000192bf3c84 ReceiveNextEventCommon + 320
9   HIToolbox                           0x0000000192bf3b2c _BlockUntilNextEventMatchingListInModeWithFilter + 72
10  AppKit                              0x000000018c799838 _DPS

/Users/Joseph/Desktop/N2_parafilm_micropub/ByPlate
done step 2


# 3. User Defined Variables (Add input here)

Here, we add some constants to help you blaze through this code.

3.1: Number of taps is pretty self-explanatory. How any taps does your experiment have? put in that number + 1 (N+1)!

This may be a bit confusing, but this is due to some coding syntax that you don't have to worry about.

3.2: Change your ISI number. This will be reflected in the name/title of the output figure.

In [3]:

number_of_taps = 30 ###<------ Taps in your experiment.

number_taps = range(1, number_of_taps + 1)  #<- do not change this


# if you have different ISIs in the same folder, then come back and change this 
# when you are graphing for the second set of data with the other ISI 
# (Generally data from same ISIs are graphed together)
# If changing ISI mid-analysis, you can just skip straight to step 8 after running this cell again

ISI = 10  ### <- What is your ISI? change accordingly
first_tap = 600 ### <- when is your first tap? check your TRV files

#Here, open up one of the trv files to determine the times for each of these taps. 
lower = np.arange(first_tap-2.5, first_tap-2.5+(number_of_taps*ISI), ISI) #(first tap, last tap+10s, ISI)
upper = np.arange(first_tap+2.5, first_tap+2.5+(number_of_taps*ISI), ISI) #(first tap, last tap+10s, ISI)
tolerances = list(zip(lower, upper)) 
taps = [i for i in range(1,number_of_taps+1)]

#assign each tolerance to a tap number

accurate_taps = list((zip(taps, tolerances)))
print(accurate_taps)

for pair in accurate_taps:
   tap = pair[0]
   tolerance = pair[1]
   #print("tap "+str(tap), "tolerance="+str(tolerance))

print("done step 3")

[(1, (597.5, 602.5)), (2, (607.5, 612.5)), (3, (617.5, 622.5)), (4, (627.5, 632.5)), (5, (637.5, 642.5)), (6, (647.5, 652.5)), (7, (657.5, 662.5)), (8, (667.5, 672.5)), (9, (677.5, 682.5)), (10, (687.5, 692.5)), (11, (697.5, 702.5)), (12, (707.5, 712.5)), (13, (717.5, 722.5)), (14, (727.5, 732.5)), (15, (737.5, 742.5)), (16, (747.5, 752.5)), (17, (757.5, 762.5)), (18, (767.5, 772.5)), (19, (777.5, 782.5)), (20, (787.5, 792.5)), (21, (797.5, 802.5)), (22, (807.5, 812.5)), (23, (817.5, 822.5)), (24, (827.5, 832.5)), (25, (837.5, 842.5)), (26, (847.5, 852.5)), (27, (857.5, 862.5)), (28, (867.5, 872.5)), (29, (877.5, 882.5)), (30, (887.5, 892.5))]
done step 3


# 4. Constructing Filelist From Source File/Select File (Just run)

In [4]:
#folder_path = '/Users/Joseph/Desktop/AVR14_10sISI' #- manual folder path if Tkinter is acting up

os.chdir(folder_path) #<- setting your working directory so that your images will be saved here

filelist = list() #<- empty list
for root, dirs, files in os.walk(folder_path): #<- this for loop goes through your folder 
    for name in files:
        if name.endswith('.trv'): #<- and takes out all files with a .trv (file that contains your data)
            filepath = os.path.join(root, name) #<- Notes down the file path of each data file
            filelist.append(filepath) #<- saves it into the list
    
print(filelist)
print('done step 4')

['/Users/Joseph/Desktop/N2_parafilm_micropub/ByPlate/Yes_Parafilm/20220820_093722/N2_P_8_10x2_t96h20C_600s30x10s_B_2022_08_20.trv', '/Users/Joseph/Desktop/N2_parafilm_micropub/ByPlate/Yes_Parafilm/20220820_090154/N2_P_2_10x2_t96h20C_600s30x10s_B_2022_08_20.trv', '/Users/Joseph/Desktop/N2_parafilm_micropub/ByPlate/Yes_Parafilm/20220820_101520/N2_7_10x2_t96h20C_600s30x10s_A_2022_08_20.trv', '/Users/Joseph/Desktop/N2_parafilm_micropub/ByPlate/Yes_Parafilm/20220820_085200/N2_P_3_10x2_t96h20C_600s30x10s_C_2022_08_20.trv', '/Users/Joseph/Desktop/N2_parafilm_micropub/ByPlate/Yes_Parafilm/20220820_093953/N2_P_1_10x2_t96h20C_600s30x10s_A_2022_08_20.trv', '/Users/Joseph/Desktop/N2_parafilm_micropub/ByPlate/Yes_Parafilm/20220913_143423/N2_P_1_10x2_t96h20C_600s30x10s_C_2022_09_13.trv', '/Users/Joseph/Desktop/N2_parafilm_micropub/ByPlate/Yes_Parafilm/20220913_144636/N2_P_2_10x2_t96h20C_600s30x10s_B_2022_09_13.trv', '/Users/Joseph/Desktop/N2_parafilm_micropub/ByPlate/Yes_Parafilm/20220820_092820/N2_

# 5. Process Data Function (Just Run)

In [11]:
def ProcessData(strain): #<- an example of a user-defined function
    strain_filelist = [x for x in filelist if strain in x] #<- goes through the list and filters for keyword
    Strain_N = len(strain_filelist) #<- N per strain, or number of plates
    Plate_N = 1
    print(f'this strain/treatment has {Strain_N} plates') #<- will output as the first number
    if Strain_N == 0:
        raise AssertionError ('{} is not a good identifier'.format(strain))
    else:
        pass
        try:
            DF_Total = pd.concat([pd.read_csv(f, sep=' ', skiprows=0, header=None) for f in strain_filelist], ignore_index=True)  # <- imports your data files
        except:
            DF_Total = pd.concat([pd.read_csv(f, sep=' ', skiprows=4, header=None) for f in strain_filelist], ignore_index=True)
        else:
            DF_Total = pd.concat([pd.read_csv(f, sep=' ', skiprows=0, header=None) for f in strain_filelist],ignore_index=True)
#     for f in strain_filelist:
#         DF_Total = pd.concat(pd.read_csv(f, sep=' ', skiprows = 4, header = None))

#         DF_Total = pd.concat([pd.read_csv(f, sep=' ', header = None) for f in strain_filelist],
#                       ignore_index=True) #<- imports your data files

        DF_Total = DF_Total.dropna(axis = 1) #<- cleans your data
        DF_Total = DF_Total.rename( #<- more cleaning
                    {0:'time',
                    2:'rev_before',
                    3:'no_rev',
                    4:'stim_rev',
                    7:'dist',
                    8:'dist_std',
                    9:'dist_stderr',
                    11:'dist_0th',
                    12:'dist_1st',
                    13:'dist_2nd',
                    14:'dist_3rd',
                    15:'dist_100th',
                    18:'dura',
                    19:'dura_std',
                    20:'dura_stderr',
                    22:'dura_0th',
                    23:'dura_1st',
                    24:'dura_2nd',
                    25:'dura_3rd',
                    26:'dura_100th'}, axis=1)
        #check function here for NaN Columns
        DF_Total['plate'] = 0
        DF_Total['prob'] = DF_Total['stim_rev']/ (DF_Total['no_rev'] + DF_Total['stim_rev']) #<- calculate prob
        DF_Total['speed'] = DF_Total['dist']/DF_Total['dura'] #<- calculate speed
        DF_Total_rows = int(DF_Total.shape[0])
        print(f'this strain/treatment has {DF_Total_rows} total taps') #<- Outputs as the second number. Check if you are missing taps!
        DF_Final = DF_Total[["time", "dura", "dist", "prob", "speed", "plate"]].copy()

    return{
            'N': Strain_N,
            'Confirm':DF_Total,
            'Final': DF_Final}
    print('done!')



def assign_taps(DF, tolerances):
    DF['taps'] = ""
    for taps, tolerance in enumerate(tolerances): #[(99, 101), (109,111), ...]
        tap_lower,tap_upper = tolerance
        TimesInTapRange = DF['time'].between(tap_lower,tap_upper, inclusive='both')
        DF.loc[TimesInTapRange,'taps'] = taps+1 #set the tap to i where times are between



    
def insert_plates(df):   
    """This function inserts a plate column into a dataframe.
    
    :param df: any dataframe
    :type: pandas.core.frame.DataFrame
    
    :return: dataframe with a plate column
    :type: pandas.core.frame.DataFrame
    """
    df['plate']=(df['taps'] ==1).cumsum()


            
print('done step 5')

done step 5


# 6.1 Process Data (PLEASE READ, Add input here)

Here is the hardest part - From your naming convention, you need to pick a unique identifier for each group.

This means that all of names of your files for that strain should have that in common but is not commone with across all other files! If you did a good job naming your files and following a good naming convention, this should be easy.

## Be careful and really look hard in your naming structure. Note you want an unique identifier in the entire file path for the same group of files. An easy mistake is to have the strain name in the overall folder name, in this case if you use your strain name as a keyword it would include all files in that folder!

For example, if all your N2 files have a certain pattern like "N2_5x4" in this following example:
'/Users/Joseph/Desktop/AVR14_10sISI_TapHab_0710_2019/N2/20190710_141740/N2_5x4_f94h20c_100s30x10s10s_C0710ab.trv'

Then you need to set that identifier for the strain keyword:
'Strain_1' = 'N2_5x4'

## Depending on how many strains you are running for comparison, you may need to add/delete some lines!

## You are not naming your data groups here, we have a step for that later!
## Here, you want to note down ALL the strains you have in the folder

If you have just 2 strains, add hashtags (#) in front of the lines you do not need.
If you need more strains, just add more lines, following the same format!

In [12]:
#Format: 'Strain_#'' = 'unique_identifier'

### Make into dictionary
StrainNames = {
    'Strain_1' : 'No_Parafilm',  #<- each strain will be designated to a unique identifier here
    'Strain_2' : 'Yes_Parafilm',
    'Strain_3' : '',
    'Strain_4' : '',
    'Strain_5' : '',
    'Strain_6' : '',
    'Strain_7' : '', #<- empty entries are for those hardcore trackers that tracks this many strains
    'Strain_8' : '',
    'Strain_9' : '',
    'Strain_10' : '',
    'Strain_11' : '',
    'Strain_12' : '',
    'Strain_13' : '',
    'Strain_14' : '',
    'Strain_15' : '',}
#...etc, etc

print('done step 6.1')

done step 6.1


# 6.2 Process Data (just run this cell)

In [13]:
DataLists = [0]  #<- generates empty list. 0 is there to account for python's index starting at 0. 
# we want indexing to start at 1 (when I say #1 I want the first point, not the second point)

for s in StrainNames.values(): #<- goes through the dictionary in step 6.1 and processes data
    if not s == '':
        DataLists.append(ProcessData(s)['Final']) #<- appends all data into a list of dataframes


for df in DataLists[1:]: 
    assign_taps(df, tolerances)
for df in DataLists[1:]:    
    insert_plates(df)

# print(x)
print(DataLists[2])
#print(len(DataLists))        
print('done step 6.2')

this strain/treatment has 18 plates
this strain/treatment has 540 total taps
this strain/treatment has 18 plates
this strain/treatment has 540 total taps
        time  dura   dist      prob     speed  plate taps
0    599.930  2.73  0.626  0.787879  0.229304      1    1
1    609.929  2.65  0.568  0.805556  0.214340      1    2
2    619.962  2.33  0.586  0.756757  0.251502      1    3
3    629.958  1.62  0.371  0.767442  0.229012      1    4
4    639.945  1.72  0.361  0.585366  0.209884      1    5
..       ...   ...    ...       ...       ...    ...  ...
535  849.970  1.63  0.287  0.181818  0.176074     18   26
536  859.931  1.70  0.424  0.219512  0.249412     18   27
537  869.971  1.71  0.329  0.225000  0.192398     18   28
538  879.970  1.55  0.332  0.302326  0.214194     18   29
539  889.928  1.58  0.392  0.250000  0.248101     18   30

[540 rows x 7 columns]
done step 6.2


# 7. Grouping Data and Naming (Optional: Add input here)

Here, you get to name your data groups/strain! Name your groups however you like under between the quotation marks for each strain.

For example: If your Strain1 is N2 and you wish for the group to be called N2,
your line should look like:

DataLists[x].assign(dataset = 'N2')

## Go back to step 6.1 to check which strain is which item on the DataLists.
In this example, the first item on DataLists is N2.


## Remember: Put your name in quotes. (ex: 'N2' and not N2)

As default, the names are set to the unique identifier labels.

## Depending on the number of strains you are running the comparison, you may have to delete/add lines of code (following the same format). 
## Remember to add/delete commas too.

# If you want to change your groups, you do that here. 
For example, if you have 5 strains in your folder but only want to compare between 2 or 3 strains, designate that here and follow through with steps 6 and 7. Once you are done, come back to step 6 and change your groups again (You are going to have to change your graph titles for the second run-through though)!

In [14]:
TotalConcatenated = pd.concat([ #<- this function joins your data together with an extra column for which group 
                          DataLists[1].assign(dataset = StrainNames.get("Strain_1")), 
                          DataLists[2].assign(dataset = StrainNames.get("Strain_2"))
#                           DataLists[3].assign(dataset = StrainNames.get("Strain_3")),
#                           DataLists[4].assign(dataset = StrainNames.get("Strain_4")),
#                           DataLists[5].assign(dataset = StrainNames.get("Strain_5")),
                          
])
# ...etc etc

TotalConcatenated.reset_index(inplace=False)
print(TotalConcatenated)
TotalConcatenated.to_csv("output.csv", encoding='utf-8', index=False)
#if TotalConcatenated["taps"].loc[ind] is not 1:
#   TotalConcatenated["taps"].loc[ind:indices[c+1]] = list(range(1,len(TotalConcatenated["taps"].loc[ind:indices[c+1]])+1))
# missing_taps(TotalConcatenated, accurate_taps, tolerances)

print('done step 7')

        time  dura   dist      prob     speed  plate taps       dataset
0    599.982  3.57  0.868  0.880952  0.243137      1    1   No_Parafilm
1    609.939  2.78  0.703  0.675676  0.252878      1    2   No_Parafilm
2    619.980  2.54  0.685  0.660377  0.269685      1    3   No_Parafilm
3    629.938  2.10  0.548  0.537037  0.260952      1    4   No_Parafilm
4    639.900  1.92  0.553  0.327586  0.288021      1    5   No_Parafilm
..       ...   ...    ...       ...       ...    ...  ...           ...
535  849.970  1.63  0.287  0.181818  0.176074     18   26  Yes_Parafilm
536  859.931  1.70  0.424  0.219512  0.249412     18   27  Yes_Parafilm
537  869.971  1.71  0.329  0.225000  0.192398     18   28  Yes_Parafilm
538  879.970  1.55  0.332  0.302326  0.214194     18   29  Yes_Parafilm
539  889.928  1.58  0.392  0.250000  0.248101     18   30  Yes_Parafilm

[1080 rows x 8 columns]
done step 7


# Setting Colour Palette - Only run the below cell ONCE

The following code sets the colour palette for the whole experiment - and then designate one colour to each strain. After this, if as you are graphing you take away some strains, you can do so with the colours still matching accordingly.

In [16]:
strains = TotalConcatenated['dataset'].unique()
pal = dict(zip(strains, sns.color_palette("deep", n_colors=len(strains))))
print('palette is:' + str(pal))
print('done setting colour palette')

palette is:{'No_Parafilm': (0.2980392156862745, 0.4470588235294118, 0.6901960784313725), 'Yes_Parafilm': (0.8666666666666667, 0.5176470588235295, 0.3215686274509804)}
done setting colour palette


# 7.5: BONUS - Graph by plates to check for anomolies across each strain/treatment

Here, we will be graphing each strain on their own by their individual plates - this will help us find any anomolies or outliers that we can then exclude!


In [ ]:
Strain1_pal = sns.choose_colorbrewer_palette('sequential')
# Strain2_pal = sns.choose_colorbrewer_palette('sequential')
# Strain3_pal = sns.choose_colorbrewer_palette('sequential')
# Strain4_pal = sns.choose_colorbrewer_palette('sequential')
# Strain5_pal = sns.choose_colorbrewer_palette('sequential')
# Strain6_pal = sns.choose_colorbrewer_palette('sequential')
# Strain7_pal = sns.choose_colorbrewer_palette('sequential')
# Strain8_pal = sns.choose_colorbrewer_palette('sequential')

In [ ]:
StrainConcatenated = TotalConcatenated[
    TotalConcatenated.dataset == StrainNames.get("Strain_1")] #Change/dictate which strain you are analyzing
StrainName = StrainNames.get("Strain_1") #Also change this, for labeling purposes


out1 = widgets.Output() #<- this is part of the code that helps display your graphs in tabs
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()

tab = widgets.Tab(children = [out1, out2, out3, out4]) #<- more code to support tab display
tab.set_title(0, 'Probability')
tab.set_title(1, 'Distance')
tab.set_title(2, 'Duration')
tab.set_title(3, 'Speed')
display(tab)

plt.rcParams.update({'lines.markeredgewidth': 1})

# Probability
with out1:
    plt.figure(linewidth = 2.5) #<- Make your figure lines THICCCC for clean display (posters)
    plt.figure(figsize=(12,10)) #<- set your figure size (width, length)
    plt.gca().xaxis.grid(False) #<- gets rid of x-axis markers to make data look clean 
    ax = sns.pointplot(x="taps", #<- Here we use seaborn as our graphing package. 
                 y="prob",  
                 data = StrainConcatenated,
                 hue = 'plate', #<- Here we use the extra column from step 6 to separate by group
                 palette = Strain1_pal, #<- Change colour palette here if you like
                 ci = 95) #<- Confidence interval. 95 = standard error
    plt.xlabel("Taps") #<- X-axis title
    plt.ylabel("Probability") #<- Y-Axis title
    plt.title(f"{StrainName} Probability of Tap Habituation, {ISI}ISI") #<- Figure Title
    plt.ylim(0,1)
    ax.legend(loc = 'upper right', fontsize = '12') #<- location of your legend
#     plt.savefig(f'{StrainName}_Probability_{ISI}ISI_ByPlate.png', format='png', dpi=900) #<- saves your file to your folder at certain DPI
    plt.show()

# Distance
with out2:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.pointplot(x="taps", 
                 y="dist", 
                 data = StrainConcatenated,
                 hue = 'plate',
                 palette = Strain1_pal,
                 ci = 95)
    plt.xlabel("Taps", fontsize = '12')
    plt.ylabel("Distance", fontsize = '12')
    plt.title(f"{StrainName} Distance of Tap Habituation, {ISI}ISI", fontsize = '16')
    plt.ylim(0,1.5)
    ax.legend(loc = 'upper right', fontsize = '12')
#     plt.savefig(f'{StrainName}_Distance_{ISI}ISI_ByPlate.png', format='png', dpi=900)
    plt.show()

# Duration
with out3:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.pointplot(x="taps", 
                 y="dura", 
                 data = StrainConcatenated,
                 hue = 'plate',
                 palette = Strain1_pal,
                 ci = 95)
    plt.xlabel("Taps", fontsize = '12')
    plt.ylabel("Duration", fontsize = '12')
    plt.title(f"{StrainName} Duration of Tap Habituation, {ISI}ISI", fontsize = '16')
    plt.ylim(0,3)
    ax.legend(loc = 'upper right', fontsize = '12')
#     plt.savefig(f'{StrainName}_Duration_{ISI}ISI_ByPlate.png', format='png', dpi=900)
    plt.show()

# Speed
with out4:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.pointplot(x="taps", 
                 y="speed", 
                 data = StrainConcatenated,
                 hue = 'plate',
                 palette = Strain1_pal,
                 ci = 95)
    plt.xlabel("Taps", fontsize = '12')
    plt.ylabel("Speed", fontsize = '12')
    plt.title(f"{StrainName} Speed of Tap Habituation, {ISI}ISI", fontsize = '16')
    plt.ylim(0,0.5)
    ax.legend(loc = 'upper right', fontsize = '12')
#     plt.savefig(f'{StrainName}_Speed_{ISI}ISI.png_ByPlate', format='png', dpi=900)
    plt.show()

# 8. Graph Data (Probability, Distance, Duration, Speed of Tap Habituation)

## Here, feel free to change the graph title and axis labels! Images are automatically saved in your folder with your data!

Note: It has been agreed by the lab that distance is quite an outdated measure, and the three top measures we consider are:

## probability, duration and speed.

In [17]:
out1 = widgets.Output() #<- this is part of the code that helps display your graphs in tabs
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()

tab = widgets.Tab(children = [out1, out2, out3, out4]) #<- more code to support tab display
tab.set_title(0, 'Probability')
tab.set_title(1, 'Distance')
tab.set_title(2, 'Duration')
tab.set_title(3, 'Speed')
display(tab)

plt.rcParams.update({'lines.markeredgewidth': 1})

# Probability
with out1:
    plt.figure(linewidth = 2.5) #<- Make your figure lines THICCCC for clean display (posters)
    plt.figure(figsize=(12,10)) #<- set your figure size (width, length)
    plt.gca().xaxis.grid(False) #<- gets rid of x-axis markers to make data look clean 
    ax = sns.pointplot(x="taps", #<- Here we use seaborn as our graphing package. 
                 y="prob",  
                 data = TotalConcatenated,
                 hue = 'dataset', #<- Here we use the extra column from step 6 to separate by group
                 palette = pal, #<- Change colour palette here if you like
                 ci = 95) #<- Confidence interval. 95 = standard error
    plt.xlabel("Taps") #<- X-axis title
    plt.ylabel("Probability") #<- Y-Axis title
    plt.title(f"Probability of Tap Habituation, {ISI}ISI") #<- Figure Title
    plt.ylim(0,1)
    ax.legend(loc = 'upper right', fontsize = '12') #<- location of your legend
#     plt.savefig(f'Probability_{ISI}ISI.png', format='png', dpi=900) #<- saves your file to your folder at certain DPI
    plt.show()

# Distance
with out2:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.pointplot(x="taps", 
                 y="dist", 
                 data = TotalConcatenated,
                 hue = 'dataset',
                 palette = pal,
                 ci = 68  #ci=68 for sem
                      )
    plt.xlabel("Taps", fontsize = '12')
    plt.ylabel("Distance", fontsize = '12')
    plt.title(f"Distance of Tap Habituation, {ISI}ISI", fontsize = '16')
    plt.ylim(0,None)
    ax.legend(loc = 'upper right', fontsize = '12')
#     plt.savefig(f'Distance_{ISI}ISI.png', format='png', dpi=900)
    plt.show()

# Duration
with out3:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.pointplot(x="taps", 
                 y="dura", 
                 data = TotalConcatenated,
                 hue = 'dataset',
                 palette = pal,
                 ci = 68  #ci=68 for sem
                      )
    plt.xlabel("Taps", fontsize = '12')
    plt.ylabel("Duration", fontsize = '12')
    plt.title(f"Duration of Tap Habituation, {ISI}ISI", fontsize = '16')
    plt.ylim(0,None)
    ax.legend(loc = 'upper right', fontsize = '12')
#     plt.savefig(f'Duration_{ISI}ISI.png', format='png', dpi=900)
    plt.show()

# Speed
with out4:
    plt.figure(linewidth = 2.5)
    plt.figure(figsize=(12,10))
    plt.gca().xaxis.grid(False)
    ax = sns.pointplot(x="taps", 
                 y="speed", 
                 data = TotalConcatenated,
                 hue = 'dataset',
                 palette = pal,
                 ci = 68 #ci=68 for sem
                      )
    plt.xlabel("Taps", fontsize = '12')
    plt.ylabel("Speed", fontsize = '12')
    plt.title(f"Speed of Tap Habituation, {ISI}ISI", fontsize = '16')
    plt.ylim(0,None)
    ax.legend(loc = 'upper right', fontsize = '12')
#     plt.savefig(f'Speed_{ISI}ISI.png', format='png', dpi=900)
    plt.show()

# Done!